In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '1'

import logging
import numexpr as ne
import numpy as np
import torch
import datetime
from ddopai.envs.pricing.dynamic import DynamicPricingEnv
from ddopai.envs.pricing.dynamic_inventory import DynamicPricingInvEnv
from ddopai.envs.actionprocessors import ClipAction, RoundAction

from ddopai.experiments.experiment_functions_online import run_experiment
from ddopai.experiments.meta_experiment_functions import *
import requests
import yaml
import re
import pandas as pd
import wandb
from copy import deepcopy
import warnings
import gc
from mushroom_rl import core 
import pickle
from tqdm import tqdm, trange

In [2]:
logging_level = logging.INFO
logging.basicConfig(level=logging_level)

ne.set_num_threads(1)
torch.backends.cudnn.enabled = False
torch.set_num_threads(1)

set_warnings(logging.INFO) # turn off warnings for any level higher or equal to the input level

project_name = "CMDP-Bandit"
config_hp_sweep = import_config("config_hp_sweep.yaml")
config_env = import_config("config_env.yaml")

INFO:root:Configuration file 'config_hp_sweep.yaml' successfully loaded.
INFO:root:Configuration file 'config_env.yaml' successfully loaded.


In [3]:
artifacts = []

run = wandb.init(
    project=project_name,
    name = f"{project_name}_artifact_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
)
for artifact_index in trange(config_hp_sweep["num_trials"]):
    
    raw_data, val_index_start, test_index_start = get_online_data(
            config_env,
            overwrite=False
        )
    
    with open('data/raw_data.pkl', 'wb') as f:
        pickle.dump(raw_data, f)
    seed = np.random.randint(0, 2**32 - 1)
    artifact = wandb.Artifact('raw_data', type='data')
    artifact.add_file('data/raw_data.pkl')
    artifact.metadata = {
        "seed": seed
    }
    wandb.log_artifact(artifact)
    artifact.wait()
    artifacts.append(artifact.name)
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: timlachner. Use `wandb login --relogin` to force relogin


100%|██████████| 1/1 [00:04<00:00,  4.88s/it]


In [4]:
config_hp_sweep["parameters"]["artifacts"] = {"values": [artifacts]}


In [5]:
del config_hp_sweep["num_trials"]
config_hp_sweep

{'method': 'random',
 'metric': {'name': 'mean_true_reward', 'goal': 'maximize'},
 'parameters': {'config_train-agent': {'values': ['RL2PPO']},
  'config_agent-RL2PPO-learning_rate_actor': {'distribution': 'log_uniform_values',
   'min': '1e-5',
   'max': '1e-3'},
  'config_agent-RL2PPO-learning_rate_critic': {'distribution': 'log_uniform_values',
   'min': '1e-5',
   'max': '1e-3'},
  'config_agent-RL2PPO-gamma': {'values': [0.0]},
  'config_agent-RL2PPO-lam': {'distribution': 'uniform',
   'min': 0.9,
   'max': 0.999},
  'config_agent-RL2PPO-eps_ppo': {'distribution': 'uniform',
   'min': 0.1,
   'max': 0.5},
  'config_agent-RL2PPO-n_epochs_policy': {'values': [1, 2, 3]},
  'config_agent-RL2PPO-ent_coeff': {'distribution': 'log_uniform_values',
   'min': '1e-8',
   'max': '1e-3'},
  'config_agent-RL2PPO-meta_episodes_per_policy_update': {'values': [1, 2, 4]},
  'config_agent-RL2PPO-meta_episodes_per_learner_batch': {'values': [8,
    16,
    32]},
  'config_agent-RL2PPO-hidden_layers

In [6]:
sweep_id = wandb.sweep(config_hp_sweep, project=project_name)

Create sweep with ID: r277nxjf
Sweep URL: https://wandb.ai/timlachner/CMDP-Bandit/sweeps/r277nxjf


In [7]:
wandb.finish()